In [53]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from data.paths.parquet_paths import BUSINESS
from pandas_setup.pandas_setup import get_parquet_pandas

In [54]:
business_df = get_parquet_pandas(str(BUSINESS))
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   10000 non-null  object 
 1   name          10000 non-null  object 
 2   address       10000 non-null  object 
 3   attributes    8590 non-null   object 
 4   city          10000 non-null  object 
 5   state         10000 non-null  object 
 6   postal_code   10000 non-null  object 
 7   latitude      10000 non-null  float32
 8   longitude     10000 non-null  float32
 9   stars         10000 non-null  float32
 10  review_count  10000 non-null  int32  
 11  is_open       10000 non-null  int32  
 12  categories    10000 non-null  object 
 13  hours         7716 non-null   object 
dtypes: float32(3), int32(2), object(9)
memory usage: 898.6+ KB


In [55]:
business_df.isna().sum()

business_id        0
name               0
address            0
attributes      1410
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
categories         0
hours           2284
dtype: int64

In [56]:
business_df['business_id'].is_unique
business_df = business_df.set_index('business_id')

In [57]:
business_df[['stars', 'review_count']].describe()

,stars,review_count
count,10000.00000,10000.000000
mean,3.55045,35.067600
std,1.01852,103.118055
min,1.00000,3.000000
25%,3.00000,4.000000
50%,3.50000,9.000000
75%,4.50000,26.000000
max,5.00000,4138.000000


In [58]:
business_df.sample(1)

,name,address,attributes,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
business_id,,,,,,,,,,,,,
oRBIgTHfNDritzEP5pvfRQ,Bank of America,1910 W Thunderbird Rd,"{'AcceptsInsurance': None, 'AgesAllowed': None...",Phoenix,AZ,85023,33.611488,-112.100311,1.5,7,1,"[Mortgage Brokers, Financial Services, Banks &...",None


In [59]:
def append_nested_columns(dataframe: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Flattens nested json and appends result columns to target dataframe
    :param dataframe: target pandas dataframe
    :param col_name: name of column containing json
    :returns: targe df with appended nested columns
    """
    dataframe = pd.concat([dataframe, pd.json_normalize(dataframe[col_name].values.tolist())], axis=1)
    return dataframe

In [60]:
business_df = append_nested_columns(business_df, 'attributes')
business_df = business_df.drop('attributes', axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, iBof5Tl8vrKISbi82tFEvQ to 9999
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        10000 non-null  object 
 1   address                     10000 non-null  object 
 2   city                        10000 non-null  object 
 3   state                       10000 non-null  object 
 4   postal_code                 10000 non-null  object 
 5   latitude                    10000 non-null  float32
 6   longitude                   10000 non-null  float32
 7   stars                       10000 non-null  float32
 8   review_count                10000 non-null  float64
 9   is_open                     10000 non-null  float64
 10  categories                  10000 non-null  object 
 11  hours                       7716 non-null   object 
 12  AcceptsInsurance            413 non-null    object 
 13  AgesAllowed     

In [61]:
has_wifi = ['paid', 'free']
business_df['WiFi'] = business_df['WiFi'].apply(lambda x: x if x in has_wifi else 'no')
business_df['WiFi'].unique()

array(['no', 'free', 'paid'], dtype=object)

In [62]:
%store business_df

Stored 'business_df' (DataFrame)


In [63]:
path = Path.cwd().joinpath('..', 'json-data', 'business.json').resolve()
business_df.to_json(path_or_buf=path, orient="records", index=True)
# business_df.to_json(path_or_buf=path, orient="split")